In [ ]:
import importlib
import datetime

import pandas as pd
import yt.wrapper as yt
from nile.api.v1 import Record
from qb2.api.v1 import typing as qt

from projects.branding.antifraud_database import project_config
from projects.branding.antifraud_database import default_values
from projects.branding.antifraud_database.nirvana import write_exams
from projects.branding import common as common_br
from projects.common.nile import test_utils

In [ ]:
pd.options.display.max_columns = 100

In [ ]:
importlib.reload(default_values)
importlib.reload(write_exams)

In [ ]:
yt.config['proxy']['url'] = 'hahn.yt.yandex.net'

In [ ]:
cluster = project_config.get_project_cluster()

In [ ]:
RECORD_DICT = {
		'car_number': 'A999AA777',
		'exam_dttm': '2020-01-01T00:00:00',
		'fraud_flag': True,
		'block_start_dttm': '2020-01-02T00:00:00',
		'block_end_dttm': '2020-01-16T00:00:00',
		'block_reason': 'fraud_sticker',
		'block_tanker_key': 'dkb_temp_block',
		'exam_ticket': 'TAXIML-1658',
	}

In [ ]:
records = [Record.from_dict(RECORD_DICT)]*10

In [ ]:
for i, record in enumerate(records):
    
    write_exams.validate_record(record, i)

### Make fake table

In [ ]:
df = common_br.decode_dataframe(cluster.read(
    '//home/taxi_ml/dev/real_time_branding_control/pilot_tests/recognition_results/confirmed/fraud/branding-control-ros_2020-06-24'
).as_dataframe())

In [ ]:
df.head()

In [ ]:
df['city'].value_counts()

In [ ]:
city_mapping = {
    'saint_petersburg': 'Санкт-Петербург',
    'moscow': 'Москва',
    'rostov-on-don': 'Ростов-на-Дону',
    'omsk': 'Омск',
}

In [ ]:
df[df['city']=='samara']['probability'].value_counts()

In [ ]:
df[df['city']=='omsk']['probability'].value_counts()

In [ ]:
df[df['city']=='rostov-on-don']['probability'].value_counts()

In [ ]:
df[df['city']=='saint_petersburg']['probability'].value_counts()

In [ ]:
records = []
for row in df.itertuples():
    if row.city in ['samara', 'xavier_imp_telegram_bot']:
        continue
    record_dict = {}
    record_dict['car_number'] = row.number_normalized
    record_dict['car_number_normalized'] = row.number_normalized
    record_dict['exam_dttm'] = row.datetime_local.replace(' ', 'T')
    record_dict['fraud_sticker_flag'] = 'sticker' in row.reasons
    record_dict['fraud_lightbox_flag'] = 'lightbox' in row.reasons
    record_dict['block_start_dttm'] = row.datetime_local.replace(' ', 'T')
    record_dict['block_end_dttm'] = (
        (datetime.datetime.strptime(row.datetime_local.replace(' ', 'T'),
                                    default_values.YT.DTTM_FORMAT)
         + datetime.timedelta(days=14)).strftime(default_values.YT.DTTM_FORMAT)
    )
    record_dict['block_reason'] = 'rostelekom_cameras'
    record_dict['block_tanker_key'] = 'dkb_temp_block'
    record_dict['exam_ticket'] = 'TAXIML-1658'
    record_dict['city'] = city_mapping.get(row.city, row.city)
    record_dict['responsible_staff_login'] = 'stleznev'
    record_dict['source_specific_attributes'] = {'photo_link': row.image}
    records.append(Record.from_dict(record_dict))

In [ ]:
row.address

In [ ]:
len(records)

In [ ]:
cluster.write('//home/taxi-fraud/import/production/qc_photos/branding/antifraud_database/test_upload',
              records)

In [ ]:
# fraud_park_set = set(df['park_id'].value_counts()[df['park_id'].value_counts() > 10].index)

In [ ]:
# for key, group in df[df['park_id'].apply(lambda x: x in fraud_park_set)].groupby('park_id'):
#     for row in group.itertuples():
#         print(row.reasons)
#     print('')

In [ ]:
# records_parks = []
# for park_id in fraud_park_set:
#     record = Record.from_dict({
#         'park_id': '',
#         'city': 'saint-petersburg',
#         'exam_dttm': '2020-06-07T00:00:00',
#         'fraud_sticker_flag': True,
#         'fraud_lightbox_flag': True,
#         'block_start_dttm': '2020-06-08T00:00:00',
#         'block_end_dttm': '2020-06-22T00:00:00',
#         'block_reason': 'park_exam',
#         'block_tanker_key': 'unedfined',
#         'exam_ticket': 'TAXIBRANDINGOPS-311',
#         'modified_dttm': '2020-06-09T18:00:00',
#     })
#     records_parks.append(record)

In [ ]:
# cluster.write(default_values.YT.PARK_LOG_PATH, records_parks, append=True)

In [ ]:
df_ids = cluster.read('//home/taxi_ml/dev/branding/antifraud_database/block_ids').as_dataframe()

In [ ]:
df_ids['source']

In [ ]:
df_ids['park_id'].value_counts()

In [ ]:
df_ids['sticker_assumed'].value_counts()

In [ ]:
df_ids['lightbox_assumed'].value_counts()

In [ ]:
((df_ids['lightbox_assumed']) & (df_ids['sticker_assumed'])).value_counts()

### Test hitman

In [ ]:
import requests

def hitman_process_starter():
    """hitman process starter"""
    headers = {
        'Content-Type': 'application/json',
        'Authorization': ''
        }

    hitman = 'https://hitman.yandex-team.ru/'
    start_proc = 'api/v1/execution/start/'
    service = "score_drivers"
    start_proc_url = hitman + start_proc + service

    data = {}
#     data['requester'] = 'stleznev'
    data['requester'] = 'robot-taxi-rqc'
    data['properties'] = {}

    resp = requests.post(start_proc_url, headers=headers, json=data, verify=False)
    print(resp.text)

In [ ]:
hitman_process_starter()

### Test action log